# The Netflix Formula: What Drives High Ratings and Viewer Engagement
**Emily Nguyen**

### Introduction
---
Netflix is a world-wide streaming service that allows users to watch a wide range of movies and TV shows on demand. This analysis explores patterns in Netflix's catalog and what factors contribute to the popularity and ratings of Netflix content. 

**Research Question(s)**:
- **Content Type**: Do movies or TV shows generally receive higher ratings and more engagement?
    - For Viewers: Discover whether movies or shows tend to be better-rated, which can help them decide what to watch if they want something with better reviews.
    - For Netflix: Invest more in the format that drives higher ratings/engagement (e.g., do big-budget movies do better or netflix original shows?)
- **Audience**: How do age certifications (PG, R, TV-MA, etc.) relate to content popularity and ratings?
    - For Viewers: Can understand which age-rated content is most popular or possibly pick content that aligns with what's trending in their age group.
    - For Netflix: Tailor content strategy by certification and which kind of content has the most positive feedback. 
- **Trends Over Time**: How has the popularity or average rating of Netflix releases changed by release year?
    - For Viewers: Can find new movies/shows to watch based on whether older or newer content has higher ratings. 
    - For Netflix: Spot whether newer content is over- or under-performing. 

**Intended Audience**:
- Who it's for: Netflix viewers and decision makers.
- What they'll learn: Which types of content performs best and why.

### Dataset Description
---
This analysis uses the **Netflix TV Shows and Movies Dataset**, available on Kaggle, a Google-owned platform offering open datasets: https://www.kaggle.com/datasets/thedevastator/netflix-imdb-scores?resource=download. It contains **5283** unique values, where each row represents a Netflix show or movie. There are **11 columns** in total, including several variables that may help explain patterns in which Netflix titles get the most attention from viewers, and what factors influence their popularity. Here is a brief description of the variables in this dataset:
- **title**: The name of a TV show or movie. (Text)
- **type**: Indicates whether an entry is a TV show or a movie. (Text)
- **description**: A brief summary or description of a TV show or movie. (Text)
- **release_year**: The year in which a TV show or movie was released. (Numeric)
- **age_certification**: The age certification rating for a TV show or movie (Text)
- **runtime**: The length of an episode for TV shows or duration of a movie. (Numeric)
- **imdb_score**: The score given by users on IMDB for a particular title. (Numeric)
- **imdb_votes**: The number of votes received by each title on IMDB. (Numeric)

In [1]:
import pandas as pd
# importing dataset and creating DataFrame
df = pd.read_csv("Netflix TV Shows and Movies.csv")
df.head()

,index,id,title,type,description,release_year,age_certification,runtime,imdb_id,imdb_score,imdb_votes
0,0,tm84618,Taxi Driver,MOVIE,A mentally unstable Vietnam War veteran works ...,1976,R,113,tt0075314,8.3,795222.0
1,1,tm127384,Monty Python and the Holy Grail,MOVIE,"King Arthur, accompanied by his squire, recrui...",1975,PG,91,tt0071853,8.2,530877.0
2,2,tm70993,Life of Brian,MOVIE,"Brian Cohen is an average young Jewish man, bu...",1979,R,94,tt0079470,8.0,392419.0
3,3,tm190788,The Exorcist,MOVIE,12-year-old Regan MacNeil begins to adapt an e...,1973,R,133,tt0070047,8.1,391942.0
4,4,ts22164,Monty Python's Flying Circus,SHOW,A British sketch comedy series with the shows ...,1969,TV-14,30,tt0063929,8.8,72895.0


In [2]:
# get the number of rows and columns
num_rows, num_cols = df.shape

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")

Number of rows: 5283
Number of columns: 11


**Handling Missing Values**

Before analyzing the dataset, I checked for missing values in each column. I was able to find that the 'age_certification' column had the most missing values, with 2,285 entries without an age rating. This is likely due to some older or international titles not having an assigned age rating. Given this is such a significant number, about 43% of the data, I decided to not drop the rows with missing age certification values since I could lose a lot of potentially useful information. 

Additionally, 'idmb_votes' had 16 missing values and 'description' had 5 missing values. All other columns, including 'title', 'type', 'release_year', 'runtime', and 'imdb_score', were complete. To handle these missing values:
- Titles without age_certification were kept in the dataset since other variables (like imdb_score and type) were still available for analysis.
- Missing imdb_votes or description entries were excluded from analyses.

In [3]:
# check for missing values in each column
df.isna().sum().sort_values(ascending=False)

age_certification    2285
imdb_votes             16
description             5
title                   0
id                      0
index                   0
type                    0
release_year            0
runtime                 0
imdb_id                 0
imdb_score              0
dtype: int64

In [4]:
# fill missing age_certification with 'Unknown'
df['age_certification'] = df['age_certification'].fillna('Unknown')

# drop rows where description and idmb_votes are missing
df = df.dropna(subset=['description', 'imdb_votes'])

# check for missing values in each column again
df.isna().sum().sort_values(ascending=False)

index                0
id                   0
title                0
type                 0
description          0
release_year         0
age_certification    0
runtime              0
imdb_id              0
imdb_score           0
imdb_votes           0
dtype: int64

**Data Type Conversions**

To ensure that each column in the dataset has an appropriate data type, I verified that each column had the appropriate data type, and confirmed that all the columns were correctly typed. 
- Numeric columns such as release_year, runtime, imdb_score, and imdb_votes were integer or float types to allow for arithmetic operations and plotting.
- Categorical or text-based columns, including title, type, description, age_certification, id, and imdb_id, were kept as objects (assuming strings).
- The index column is kept as an integer, serving as a unique identifier for rows in the dataset

In [5]:
df.dtypes

index                  int64
id                    object
title                 object
type                  object
description           object
release_year           int64
age_certification     object
runtime                int64
imdb_id               object
imdb_score           float64
imdb_votes           float64
dtype: object

**Filtering Unrealistic Runtimes**

During data cleaning, I also found that some titles had extremely short runtimes (0-10 minutes), which likely corresponds to data entry mistakes or trailers, shortclips, and previews rather than full episodes or movies. Including these entries could skew analyses related to popularity, ratings, and runtime. Given this, I filtered the dataset to include only titles with a runtime of 10 minutes or longer. 

In [7]:
# count rows with runtime less than 10
short_df = df[df['runtime'] < 10]
print(f"Number of titles with runtime under 10 minutes: {short_df.shape[0]}")

Number of titles with runtime under 10 minutes: 67


In [8]:
# remove rows with runtime less than 10
df = df[df['runtime'] >= 10]

num_rows, num_cols = df.shape

print(f"Number of rows: {num_rows}")
print(f"Number of columns: {num_cols}")

Number of rows: 5196
Number of columns: 11


After cleaning the dataset, the number of rows decreased from 5283 to **5196**, reflecting the removal of incomplete or unrealistic entries.

### Exploratory Data Analysis (EDA)
---
To begin, I generated summary statistics for the dataset, including measures like counts, averages, and distributions of key variables. This provided a baseline understanding of the Netflix catalog and guided which factors (e.g., content type, age certification, release year, ratings, and engagement) were worth exploring further in the visual analysis.

In [9]:
# summary statistics for numeric columns
import seaborn as sns
import matplotlib.pyplot as plt

summary_stats = df[['release_year', 'runtime', 'imdb_score', 'imdb_votes']].describe()
print(summary_stats.to_string(float_format='{:.2f}'.format))

       release_year  runtime  imdb_score  imdb_votes
count       5196.00  5196.00     5196.00     5196.00
mean        2015.86    80.28        6.53    23718.65
std            7.38    38.20        1.16    87686.56
min         1953.00    10.00        1.50        5.00
25%         2015.00    46.00        5.80      536.75
50%         2018.00    88.00        6.60     2326.00
75%         2020.00   106.00        7.40    10376.25
max         2022.00   235.00        9.60  2268288.00


### Content Type - Movies or TV Shows?
---
This section compares how movies and TV shows perform on Netflix in terms of both ratings and engagement. By examining averages and top-performing titles, the analysis highlights differences in audience reception and cultural impact between the two content types.

In [19]:
# imports
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pandas as pd

# color palette
palette = {
    "MOVIE": "#FADADD",  # pink
    "SHOW": "#DCC7F8"    # purple
}

def style_fig(fig, title):
    fig.update_layout(
        title=title,
        template="plotly_white",
        font=dict(family="Arial", size=14),
        title_font=dict(size=18, color="#333"),
        xaxis=dict(showgrid=True, gridcolor="lightgray"),
        yaxis=dict(showgrid=True, gridcolor="lightgray"),
        legend=dict(title=None, orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
        margin=dict(t=70, b=40)
    )
    return fig

In [17]:
type_counts = df["type"].value_counts().reset_index()
type_counts.columns = ["Content Type", "Count"]

fig = px.bar(
    type_counts,
    x="Content Type",
    y="Count",
    color="Content Type",
    color_discrete_map=palette,
    text="Count"
)
fig.update_traces(textposition="outside")
style_fig(fig, "Distribution of Movies vs. TV Shows on Netflix").show()



In [20]:
avg_metrics = df.groupby("type", as_index=False).agg(
    imdb_score=('imdb_score','mean'),
    imdb_votes=('imdb_votes','mean')
)

fig = make_subplots(rows=1, cols=2, subplot_titles=("Average IMDb Score", "Average IMDb Votes"))

fig.add_trace(
    go.Bar(
        x=avg_metrics["type"],
        y=avg_metrics["imdb_score"],
        marker_color=[palette[t] for t in avg_metrics["type"]],
        text=avg_metrics["imdb_score"].round(2),
        textposition="outside"
    ),
    row=1, col=1
)

fig.add_trace(
    go.Bar(
        x=avg_metrics["type"],
        y=avg_metrics["imdb_votes"],
        marker_color=[palette[t] for t in avg_metrics["type"]],
        text=avg_metrics["imdb_votes"].round(0),
        textposition="outside"
    ),
    row=1, col=2
)

fig.update_layout(template="plotly_white", title_text="Average IMDb Score and Votes by Content Type", showlegend=False)
fig.show()


In [43]:
# NEED TO FIX!
# top 5 movies and shows by IMDb score
top5_movies = df[df["type"]=="MOVIE"].nlargest(5, "imdb_score")[["title","imdb_score"]].assign(Category="Top Movies")
top5_shows = df[df["type"]=="SHOW"].nlargest(5, "imdb_score")[["title","imdb_score"]].assign(Category="Top Shows")

# bottom 5 movies and shows by IMDb score
bottom5_movies = df[df["type"]=="MOVIE"].nsmallest(5, "imdb_score")[["title","imdb_score"]].assign(Category="Bottom Movies")
bottom5_shows = df[df["type"]=="SHOW"].nsmallest(5, "imdb_score")[["title","imdb_score"]].assign(Category="Bottom Shows")

combined = pd.concat([top5_movies, top5_shows, bottom5_movies, bottom5_shows])

fig = px.bar(
    combined,
    x="imdb_score",
    y="title",
    color="Category",
    color_discrete_sequence=["#FADADD","#C9F7F5","#D6EAF8","#DCC7F8"],
    orientation="h",
    text="imdb_score"
)
fig.update_traces(texttemplate="%{text:.1f}")
fig.update_layout(template="plotly_white", yaxis_categoryorder="total ascending", title="Top & Bottom 5 Netflix Titles by IMDb Score")
fig.show()


### Audience - Age Certification
---
Here, I explore how Netflix content is distributed across age certification categories (Kids, Family/Teen, and Mature) and how these categories differ in ratings and engagement. This helps reveal which audience groups drive the most attention and satisfaction on the platform.

In [22]:
fig = px.histogram(
    df, x="age_certification", color="type",
    color_discrete_map=palette,
    barmode="group"
)
style_fig(fig, "Distribution of Age Certifications on Netflix (Including Unknown)").show()


In [23]:
df_cert_filtered = df[df['age_certification'] != "Unknown"].copy()
age_map = {
    "TV-Y": "Kids", "TV-Y7": "Kids", "TV-G": "Kids", "G": "Kids",
    "PG": "Family/Teen", "TV-PG": "Family/Teen", "PG-13": "Family/Teen", "TV-14": "Family/Teen",
    "R": "Mature", "NC-17": "Mature", "TV-MA": "Mature"
}
df_cert_filtered["age_group"] = df_cert_filtered["age_certification"].map(age_map)

fig = px.box(
    df_cert_filtered,
    x="age_group",
    y="imdb_score",
    color="type",
    color_discrete_map=palette
)
style_fig(fig, "IMDb Ratings by Audience Age Group").show()


In [24]:
avg_votes = df_cert_filtered.groupby(["age_group","type"])["imdb_votes"].mean().reset_index()

fig = px.bar(
    avg_votes,
    x="age_group",
    y="imdb_votes",
    color="type",
    color_discrete_map=palette,
    barmode="group",
    text_auto=".0f"
)
style_fig(fig, "Average IMDb Votes by Age Group and Content Type").show()


### Trends Over Time
---
This section looks at how Netflix’s catalog has evolved by release year, focusing on growth in the number of titles, shifts in ratings, and changes in audience engagement. The goal is to see whether increased production has translated into higher-quality or more widely watched content.

In [26]:
trend_df = df.groupby("release_year").agg(
    avg_score=("imdb_score", "mean"),
    avg_votes=("imdb_votes", "mean"),
    count_titles=("id", "count")
).reset_index()
trend_df = trend_df[trend_df["release_year"] >= 1990]

fig = go.Figure()
fig.add_trace(go.Bar(
    x=trend_df["release_year"], y=trend_df["count_titles"],
    name="Number of Titles", marker_color="#AED6F1", opacity=0.7, yaxis="y1"
))
fig.add_trace(go.Scatter(
    x=trend_df["release_year"], y=trend_df["avg_votes"],
    name="Average IMDb Votes", marker_color="#F5A9BC", mode="lines+markers", yaxis="y2"
))
fig.update_layout(
    template="plotly_white",
    title="Netflix Titles Released vs. Average IMDb Votes Over Time",
    xaxis=dict(title="Release Year"),
    yaxis=dict(title="Number of Titles", side="left"),
    yaxis2=dict(title="Average IMDb Votes", overlaying="y", side="right"),
    legend=dict(x=0.1, y=1.1, orientation="h")
)
fig.show()


In [28]:
fig = px.line(
    trend_df,
    x="release_year",
    y="avg_score",
    markers=True,
    color_discrete_sequence=["#AED6F1"]
)
style_fig(fig, "Average IMDb Score of Netflix Titles Over Time").show()

In [29]:
df_bubble = df[df['release_year'] >= 2000]

fig = px.scatter(
    df_bubble,
    x="release_year",
    y="imdb_score",
    size="imdb_votes",
    color="type",
    color_discrete_map=palette,
    hover_name="title",
    size_max=40,
    opacity=0.7
)
style_fig(fig, "Netflix Title Ratings and Popularity by Release Year").show()


In [31]:
# Sort by IMDb votes (biggest bubbles first)
top_titles = df_bubble.sort_values(by="imdb_votes", ascending=False)[
    ["title", "type", "release_year", "imdb_score", "imdb_votes"]
].head(15)

pd.set_option("display.float_format", "{:,.0f}".format)  # format votes without decimals

top_titles

,title,type,release_year,imdb_score,imdb_votes
246,Inception,MOVIE,2010,9,"2,268,288"
229,Breaking Bad,SHOW,2008,10,"1,727,694"
662,Django Unchained,MOVIE,2012,8,"1,472,668"
1145,Stranger Things,SHOW,2016,9,"989,090"
230,The Walking Dead,SHOW,2010,8,"945,125"
870,The Imitation Game,MOVIE,2014,8,"748,654"
248,How to Train Your Dragon,MOVIE,2010,8,"719,717"
683,Silver Linings Playbook,MOVIE,2012,8,"697,481"
262,The Social Network,MOVIE,2010,8,"681,286"
250,Casino Royale,MOVIE,2006,8,"644,336"
